In [1]:
#Start of the data cleaning process

In [2]:
#start with imports
import numpy as np
import pandas as pd

In [3]:
Advanced_df = pd.read_csv('Assets/NBA_Player_Advanced_Stats.csv')
Totals_df = pd.read_csv('Assets/NBA_Player_Total_Stats.csv')
Contracts_df = pd.read_csv('Assets/NBA_Contracts.csv')

In [4]:
Contracts_df.head()

,Signing Year,Rank,Player,Position,Team,Age,Contract Length,Total Value,Avg. Value,Guaranteed Money,Contract Type
0,2011,1,Kevin Durant,SF,OKC,22,5,"$89,163,134","$17,832,627","$89,163,134",rookie-maximum-extension
1,2011,2,Kobe Bryant,SF,LAL,32,3,"$83,547,447","$27,849,149","$83,547,447",Veteran Extension
2,2011,3,Zach Randolph,PF,MEM,29,4,"$66,000,000","$16,500,000","$66,000,000",Veteran Extension
3,2011,T4,Al Horford,C,ATL,24,5,"$60,000,000","$12,000,000","$60,000,000",Rookie Extension
4,2011,T4,Joakim Noah,C,CHI,25,5,"$60,000,000","$12,000,000","$60,000,000",Rookie Extension


In [5]:
#Totals_df.info(), Advanced_df.info()

In [6]:
player_names = Contracts_df['Player'].unique()
unique_players = len(player_names)
total_contracts = len(Contracts_df)

print(f'There are {unique_players} unique players of the {total_contracts} contracts')

There are 94 unique players of the 106 contracts


In [7]:
#Some of our players in our contracts_df have different names than what show in the stats_dfs
adv_players_unq = set(Advanced_df['Player'].unique())
total_players_unq = set(Totals_df['Player'].unique())
missing_players = [name for name in player_names if name not in adv_players_unq and name not in total_players_unq]
print(missing_players)
# Missing Players: ['Nikola Vucevic', 'Anderson Varejao', 'Marcus Morris Sr.', 'Jonas Valanciunas', 'C.J. McCollum', 'Dennis Schröder', 'Louis Williams']
#These missing players are most likely due to either nicknames(Lou Williams) or special characters(Jonas Valančiūnas)

['Nikola Vucevic', 'Anderson Varejao', 'Marcus Morris Sr.', 'Jonas Valanciunas', 'C.J. McCollum', 'Dennis Schröder', 'Louis Williams']


In [8]:
#Let's figure out our spellings within the player stats datasets
#The following code will print out a few extra names than what we need, but it will allow us to see the spellings within the total_players_unq and likely the adv_players_unq aswell
for player in total_players_unq:
    if 'Nikola Vu' in player or 'Anderson Var' in player or 'Marcus Morris' in player or 'Jonas Val' in player or 'Dennis Sch' in player or 'Lou Williams' in player or 'McCollum' in player:
        print(player)

#Sinec some of the names in the total_players_unq have some odd spelling (Schroder and Varejao) we will set those names to the name found in the COntracts_df

Marcus Morris
Lou Williams
Jonas ValanÄiÅ«nas
Nikola VuÄeviÄ
CJ McCollum
Dennis SchrÃ¶der
Anderson VarejÃ£o


In [9]:
name_corrections = {
    'Nikola VuÄeviÄ': 'Nikola Vucevic',
    'Jonas ValanÄiÅ«nas': 'Jonas Valanciunas',
    'Anderson VarejÃ£o': 'Anderson Varejao',
    'Marcus Morris': 'Marcus Morris Sr.',
    'CJ McCollum': 'C.J. McCollum',
    'Dennis SchrÃ¶der': 'Dennis Schröder',
    'Lou Williams': 'Louis Williams'
}

Advanced_df['Player'] = Advanced_df['Player'].replace(name_corrections)
Totals_df['Player'] = Totals_df['Player'].replace(name_corrections)

#Let's make sure that worked though, and make sure that both of our Basketball Reference CSVs had the same names used
advanced_players = set(Advanced_df["Player"].unique())
totals_players = set(Totals_df["Player"].unique())
contract_players = set(player_names)

unmatched_totals = contract_players - totals_players
print('Players not in totals', unmatched_totals)
unmatched_adv = contract_players - advanced_players
print('Players not in advanced', unmatched_adv)

#Our print outs show empty lists which means our originally "missing" (misspelled) names now match those used in our Contract database. 

Players not in totals set()
Players not in advanced set()


In [10]:
#Now we can filter down our stats satasets to only include the players that are in our contract data:
Advanced_df_filtered = Advanced_df[Advanced_df["Player"].isin(player_names)]
print(f"Advanced_df: {Advanced_df.shape} → {Advanced_df_filtered.shape}")

Totals_df_filtered = Totals_df[Totals_df["Player"].isin(player_names)]
print(f"Totals_df: {Totals_df.shape} → {Totals_df_filtered.shape}")

Advanced_df: (9262, 30) → (1179, 30)
Totals_df: (9262, 33) → (1179, 33)


In [11]:
#Testing:
#Advanced_df_filtered[Advanced_df_filtered["Player"] == 'Kevin Love']

# More Filtering!
### Here we will filter down our dataframe some more to filter out any rows that are not included in our Contracts_df

In [12]:
# Let's create a dict obj that will contain all the players in our contract_df with there give contract years
contract_years_dict = {}

for _,row in Contracts_df.iterrows():
    player = row['Player']
    signing_year = row['Signing Year']
    length_contract = row['Contract Length']

    #We need to make sure we keep all instances of contract signings so as we get one we will extend our dict
    #by extending we should be able to avoid any instances of overwritting earlier contracts with later ones
    #We can do this by checking if the player variable in our loop is already in our dict, and if not we will add them
    if player not in contract_years_dict:
        contract_years_dict[player] = [] 
    contract_years_dict[player].extend(range(signing_year, signing_year + length_contract)) #For the given player, add stats

#print(contract_years_dict)
#Our output looks like: {'Kevin Durant': [2011, 2012, 2013, 2014, 2015],
#                        'Kobe Bryant': [2014, 2015], 
#                        'Zach Randolph': [2015, 2016], ....

#Now that we have each player, and their contract years, we can filter our data even more to only contain their years 
#that match with the contract extensions from our Contracts_df 
filtered_totals = []
filtered_advanced = []

#For each row in our stats dataframes, we will grab the player name and season of play,
#we then check if this name and season/year is within our contract_dict. If it is we add it to our filter
for index, row in Totals_df_filtered.iterrows():
    player, year = row['Player'], row['Season']
    if player in contract_years_dict and year in contract_years_dict[player]:
        filtered_totals.append(row)

for index, row in Advanced_df_filtered.iterrows():
    player, year = row['Player'], row['Season']
    if player in contract_years_dict and year in contract_years_dict[player]:
        filtered_advanced.append(row)
        

In [13]:
#Now that we have our filter from above(as a list of lists) we can turn it back into a dataframe
#this will let us do more statistical analysis
Totals_df_years_filtered = pd.DataFrame(filtered_totals, columns=Totals_df.columns)
print(f'Totals_df: {Totals_df_years_filtered.shape}')

Advanced_df_years_filtered = pd.DataFrame(filtered_advanced, columns=Advanced_df.columns)
print(f'Advanced_df: {Advanced_df_years_filtered.shape}')

Totals_df: (489, 33)
Advanced_df: (489, 30)


In [14]:
#Let's take a look at an example. Kevin Love is a player with 2 different contracts signed
print(Advanced_df_years_filtered[Advanced_df_years_filtered["Player"] == 'Kevin Love'])

      Season   Rank      Player   Age Team Pos     G    GS      MP   PER  ...  \
928     2012  327.0  Kevin Love  24.0  MIN  PF  18.0  18.0   618.0  17.9  ...   
1143    2013   18.0  Kevin Love  25.0  MIN  PF  77.0  77.0  2797.0  26.9  ...   
1763    2014   24.0  Kevin Love  26.0  CLE  PF  75.0  75.0  2532.0  18.8  ...   
2439    2015   46.0  Kevin Love  27.0  CLE  PF  77.0  77.0  2424.0  19.0  ...   
5046    2019   96.0  Kevin Love  31.0  CLE  PF  56.0  56.0  1780.0  17.9  ...   
6023    2020  338.0  Kevin Love  32.0  CLE  PF  25.0  25.0   622.0  15.1  ...   
6475    2021  153.0  Kevin Love  33.0  CLE   C  74.0   4.0  1665.0  19.6  ...   
7378    2022  223.0  Kevin Love  34.0  2TM  PF  62.0  20.0  1240.0  13.1  ...   
7379    2022  223.0  Kevin Love  34.0  CLE  PF  41.0   3.0   821.0  13.6  ...   
7380    2022  223.0  Kevin Love  34.0  MIA  PF  21.0  17.0   419.0  12.3  ...   

      USG%   OWS  DWS    WS  WS/48  OBPM  DBPM  BPM  VORP          Honors  
928   28.9   0.1  0.9   1.1  0.0

In [15]:
#If we can now add a column variable named Age_of_contract (1,2,3,4,5) we could use this to help with
#future filtering or analysis... Then we could create new CSV's of our filtered stats 
#to load into a notebook for exploration

#We can start by creating a 
signing_years = Contracts_df.groupby('Player')['Signing Year'].apply(list)
#This gives us the Players name as index and the signing year as the value

print(f'We have our {len(signing_years)} unique players and their signing years as a list for each index')
print(signing_years['Kevin Love'])
#This example shows that if we look up KD we get his 1 year in the Contracts_df, and Kevin Love gets us 2 years

We have our 94 unique players and their signing years as a list for each index
[2012, 2019]


In [16]:
#We can use this singing_years as a way to calculate contract_age
#Let's create a function that we can apply on each of our dataframes that gets the contract_age from each row

def get_contract_age(row):
    #Who are we looking at and what year of their stats_df is it?
    player = row['Player']
    season = row['Season']

    if player in signing_years:
        possible_years = signing_years[player] #This will include all the signing years from our DF by teh given player name

        #It is important that we pick the correct year of the possible years. To do this we will want to make sure that
        #possible_year we use is <= the season value, but we also must make sure it is the MAX of any possible values
        #Kevin Love shows 2 signing years 2012 and 2019. We must use the 2019 contract if say we are looking at 2021 data

        year_signed = max([year for year in possible_years if year <= season], default=None)

        if year_signed:
            return season - year_signed + 1 
        


In [17]:
#Now we can call this against to calculate a new column variable in our dataframes
Advanced_df_years_filtered["Contract Age"] = Advanced_df_years_filtered.apply(get_contract_age, axis=1)
Totals_df_years_filtered["Contract Age"] = Totals_df_years_filtered.apply(get_contract_age, axis=1)

In [18]:
#Let's take a look at an example again. Early we say Kevin Love's contracts,
#lets make sure his contract age matches with each instance of his stats data
print(Advanced_df_years_filtered[Advanced_df_years_filtered["Player"] == 'Kevin Love'])

      Season   Rank      Player   Age Team Pos     G    GS      MP   PER  ...  \
928     2012  327.0  Kevin Love  24.0  MIN  PF  18.0  18.0   618.0  17.9  ...   
1143    2013   18.0  Kevin Love  25.0  MIN  PF  77.0  77.0  2797.0  26.9  ...   
1763    2014   24.0  Kevin Love  26.0  CLE  PF  75.0  75.0  2532.0  18.8  ...   
2439    2015   46.0  Kevin Love  27.0  CLE  PF  77.0  77.0  2424.0  19.0  ...   
5046    2019   96.0  Kevin Love  31.0  CLE  PF  56.0  56.0  1780.0  17.9  ...   
6023    2020  338.0  Kevin Love  32.0  CLE  PF  25.0  25.0   622.0  15.1  ...   
6475    2021  153.0  Kevin Love  33.0  CLE   C  74.0   4.0  1665.0  19.6  ...   
7378    2022  223.0  Kevin Love  34.0  2TM  PF  62.0  20.0  1240.0  13.1  ...   
7379    2022  223.0  Kevin Love  34.0  CLE  PF  41.0   3.0   821.0  13.6  ...   
7380    2022  223.0  Kevin Love  34.0  MIA  PF  21.0  17.0   419.0  12.3  ...   

       OWS  DWS    WS  WS/48  OBPM  DBPM  BPM  VORP          Honors  \
928    0.1  0.9   1.1  0.084   1.4  -

In [23]:
#Before beginning our analysis lets add a contract year 'flag' to the data so we can use it as a marker for a players final year of their contract
# Create a dictionary to track the last contract year for each player
final_contract_years = {}

for _, row in Contracts_df.iterrows():
    player, signing_year, length = row["Player"], row["Signing Year"], row["Contract Length"]
    
    last_contract_year = signing_year + length - 1  # Determine final season before new deal

    if player not in final_contract_years:
        final_contract_years[player] = []
    
    final_contract_years[player].append(last_contract_year) 


#now that we have our players with their final contract year we can create a function to flag the players final year as a new column
def flag_contract_year(player, season):
    if player in final_contract_years and season in final_contract_years[player]:
        return 1
    else:
        return 0

In [27]:
#Now lets apply our function and check on Kevin Love again
Totals_df_years_filtered["CY Flag"] = Totals_df_years_filtered.apply(lambda row: flag_contract_year(row["Player"], row["Season"]), axis=1)

Advanced_df_years_filtered["CY Flag"] = Advanced_df_years_filtered.apply(lambda row: flag_contract_year(row["Player"], row["Season"]), axis=1)
print(Advanced_df_years_filtered[Advanced_df_years_filtered["Player"] == 'Kevin Love']) 
# we can now see the enw variable CY flag that is 1 for contract years

      Season   Rank      Player   Age Team Pos     G    GS      MP   PER  ...  \
928     2012  327.0  Kevin Love  24.0  MIN  PF  18.0  18.0   618.0  17.9  ...   
1143    2013   18.0  Kevin Love  25.0  MIN  PF  77.0  77.0  2797.0  26.9  ...   
1763    2014   24.0  Kevin Love  26.0  CLE  PF  75.0  75.0  2532.0  18.8  ...   
2439    2015   46.0  Kevin Love  27.0  CLE  PF  77.0  77.0  2424.0  19.0  ...   
5046    2019   96.0  Kevin Love  31.0  CLE  PF  56.0  56.0  1780.0  17.9  ...   
6023    2020  338.0  Kevin Love  32.0  CLE  PF  25.0  25.0   622.0  15.1  ...   
6475    2021  153.0  Kevin Love  33.0  CLE   C  74.0   4.0  1665.0  19.6  ...   
7378    2022  223.0  Kevin Love  34.0  2TM  PF  62.0  20.0  1240.0  13.1  ...   
7379    2022  223.0  Kevin Love  34.0  CLE  PF  41.0   3.0   821.0  13.6  ...   
7380    2022  223.0  Kevin Love  34.0  MIA  PF  21.0  17.0   419.0  12.3  ...   

        WS  WS/48  OBPM  DBPM  BPM  VORP          Honors  Contract Age  \
928    1.1  0.084   1.4  -1.0  0.4

In [28]:
#This appears to work well. Now we can take these filtered datasets and turn them into our CSV files to use elsewhere
Advanced_df_years_filtered.to_csv('Assets/Player_Advanced_Filter.csv', index=False)
Totals_df_years_filtered.to_csv('Assets/Player_Totals_Filter.csv', index=False)